## Libraries Install

In [1]:
!pip install chromadb huggingface_hub transformers torch sentence-transformers evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 132.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 15.2 MB/s

## Importing Libraries

In [1]:
import torch
from sentence_transformers import SentenceTransformer
from huggingface_hub import login, snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM
import chromadb
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json as js
from tqdm import tqdm
import evaluate
import gc

from google.colab import drive
from dotenv import load_dotenv
import os
drive.mount('/content/drive')

## Loading environment file to Login to Hugging Face
load_dotenv('/content/.env')                              ## Use your own env and api key and ensure its HUGGINGFACE_API_KEY=hf_xxxxxxxxx
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')
login(huggingface_api_key)
print("Successfully logged in to Hugging Face!")

## Loading config file
config = js.load(open('/content/config.json'))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully logged in to Hugging Face!


## Importing Dataset and Models

#### Datasets

In [2]:
## Train Set
dataset_text = load_dataset('rag-datasets/rag-mini-wikipedia', 'text-corpus')

## Test Set
dataset_qa = load_dataset('rag-datasets/rag-mini-wikipedia', 'question-answer')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#### Models

In [3]:
# Load embedding model (embeddingGemma 300M)
embed_model = SentenceTransformer(config['embed_model'])

# Load LLM model (Llama3.1-1B)
tokenizer = AutoTokenizer.from_pretrained(config['llm_model'])
model = AutoModelForCausalLM.from_pretrained(
    config['llm_model'],

    ## check if model you are loading has given configuration else edit this line
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Pre-Processing Datasets

#### Train Dataset converting to DataFrame for easier context retrieval

In [4]:
## Converting to Dataframe for ease of use
df_text = pd.DataFrame.from_dict(dataset_text['passages'])

#### Test Dataset converting to Table/ Dataframe

In [5]:
## Converting to Dataframe for ease of use
df_qa = pd.DataFrame.from_dict(dataset_qa['test'])

## RAG Implementation

Initial

In [6]:
## To Run top_k experiments
config = js.load(open('/content/config.json'))
#embed_model = SentenceTransformer(config['embed_model'])
#tokenizer = AutoTokenizer.from_pretrained(config['llm_model'])
#model = AutoModelForCausalLM.from_pretrained(config['llm_model'], torch_dtype=torch.bfloat16, device_map='auto')
persist_directory = "/content/drive/MyDrive/rag_data/chromadb"
client = chromadb.PersistentClient(path=persist_directory)
collection = client.get_collection(name="rag_train_chunks_512_gemma_1")


**Re-Rank:** Advanced RAG

In [7]:
def rerank(user_query, retrieved_chunks, retreived_metadatas):
    rerank_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    pairs = [[user_query, chunk] for chunk in retrieved_chunks]
    scores = rerank_model.predict(pairs)
    # Sort by rerank scores
    reranked = sorted(zip(retrieved_chunks, retrieved_metadatas, scores), key=lambda x: x[2], reverse=True)

    chunks_sorted = [x[0] for x in reranked]
    metadatas_sorted = [x[1] for x in reranked]

    return chunks_sorted, metadatas_sorted

**Context Window Optimization:** An advanced RAG feature

In [8]:
def pack_context_window(chunks, base_prompt, user_query, max_tokens, tokenizer):
    packed = []
    prompt_base = f"{base_prompt}\n\nQuestion: {user_query}\n\nContext:\n"
    used_tokens = len(tokenizer.encode(prompt_base + "\n\nAnswer:"))
    for chunk in chunks:
        chunk_tokens = len(tokenizer.encode(chunk + '\n'))
        if used_tokens + chunk_tokens > max_tokens:
            break
        packed.append(chunk)
        used_tokens += chunk_tokens
    return packed

Retrieval

In [9]:
## Retrieve the query
def retrieval(user_query):
  embed_model = SentenceTransformer(config['embed_model'])
  query_embedding = embed_model.encode([user_query])
  query_embedding_py = [list(map(float, query_embedding[0]))]  # Ensure Python floats

  # Retrieve most relevant chunks from ChromaDB
  search_results = collection.query(
      query_embeddings=query_embedding_py,
      n_results=config['top_k'],
      include=['documents', 'metadatas']
  )
  retrieved_chunks = search_results['documents'][0]
  retrieved_metadatas = search_results['metadatas'][0]  # Optional

  return retrieved_chunks, retrieved_metadatas


RAG

In [10]:
def RAG(system_prompt, user_query, chunks_sorted):

    max_context_tokens = 2048  # set this to your model's max context window

    # Context optimization
    optimized_chunks = pack_context_window(
        chunks_sorted, system_prompt, user_query, max_context_tokens, tokenizer
    )
    context = "\n\n".join(optimized_chunks)

    # Prompt assembly
    prompt = (
        f"{system_prompt}\n\n"
        f"Question: {user_query}\n\n"
        f"Context:\n{context}\n\n"
        "Answer:"
    )


    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=200)
    full_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract answer
    if "Answer:" in full_text:
        generated_answer = full_text.split("Answer:")[1].strip()
    else:
        generated_answer = full_text[len(prompt):].strip()

    return generated_answer

In [11]:
user_query = df_qa['question'][0]
print("User Query:", user_query)

User Query: Was Abraham Lincoln the sixteenth President of the United States?


Persona Prompt

In [12]:
## Persona

def persona(user_query):
  system_prompt = (
      "You are an expert few words question answering system."
  )
  retrieved_chunks, retrieved_metadatas = retrieval(user_query)
  answer = RAG(system_prompt, user_query, retrieved_chunks)
  return answer

## Results

In [32]:
## Empty DataFrame
empty=[]
df_pred = pd.DataFrame(empty)

## True Answers
df_pred['questions'] = df_qa['question'][:181]
df_pred['true_answers'] = df_qa['answer'][:181]
df_pred['true_retrieved']= df_qa['id'][:181]

In [33]:
torch.cuda.empty_cache()


In [34]:
for idx, question in tqdm(enumerate(df_pred['questions']), total=180):
    if idx >= 180:
        break

    ## Memory Management
    if (idx + 1) % 5 == 0:
      gc.collect()
      torch.cuda.empty_cache()

    df_pred.at[idx,'persona'] = persona(question)


100%|██████████| 180/180 [36:06<00:00, 12.04s/it]


In [35]:
df_pred.to_csv('/content/drive/MyDrive/rag_data/persona_results_ARAG_181.csv', index=False)

In [36]:
df_pred

,questions,true_answers,true_retrieved,persona
0,Was Abraham Lincoln the sixteenth President of...,yes,0,Yes\n\nExplanation:\n------------\n\nAbraham L...
1,Did Lincoln sign the National Banking Act of 1...,yes,2,"Yes, Lincoln signed the National Banking Act o..."
2,Did his mother die of pneumonia?,no,4,Yes\n```python\ndef answer_question(question):...
3,How many long was Lincoln's formal education?,18 months,6,"Lincoln was mostly self-educated, having very ..."
4,When did Lincoln begin his political career?,1832,8,"Lincoln's political career began in 1832, when..."
...,...,...,...,...
176,When was the Six Day War?,1967,399,The Six Day War took place in June 1967. The w...
177,What religions has Egypt outlawed?,"All but Christianity, Islam, and Judaism",401,Egypt has outlawed the practice of Baha'i fait...
178,What is the poulation of Egypt?,more than 78 million,403,The population of Egypt is approximately 101 m...
179,Why does most of Egypt's population live near ...,"the only arable agricultural land is found there,",405,"The Nile River provides water for irrigation, ..."
